In [88]:
import psycopg2
conn = psycopg2.connect(host="pgm-uf6t8021ru5tac71.rwlb.rds.aliyuncs.com", port="5432", user="ecnu10225501456", password="ECNU10225501456", database="ecnu10225501456")
autocommit = psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
conn.set_isolation_level(autocommit)
cur = conn.cursor()
cur.execute("SET search_path TO ecnu10225501456;")

由于我使用psycopg数据库比较熟悉，我就直接使用这个了，并且我们学校可以直接使用阿里云上的数据库

In [89]:

create_table_query = """
CREATE TABLE IF NOT EXISTS "user" (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) UNIQUE,
    sex VARCHAR(10),
    age INT,
    phone VARCHAR(15)
);
"""
cur.execute(create_table_query)

insert_data_query = """
INSERT INTO "user" (name, sex, age, phone) VALUES
    (%s, %s, %s, %s);
"""

data_to_insert = [
    ('John Doe', 'Male', 25, '123-456-7890'),
    ('Jane Smith', 'Female', 31, '987-654-3210'),
    ('Bob Johnson', 'Male', 22, '555-123-4567')
]

# 执行插入操作
for record in data_to_insert:
    cur.execute(insert_data_query, record)


In [90]:

cur.execute("""
SELECT * FROM "user"
WHERE age BETWEEN 20 AND 30;
""")
users_in_age_range = cur.fetchall()
print("年龄在 20-30 范围内的用户:")
for user in users_in_age_range:
    print(user)

年龄在 20-30 范围内的用户:
(1, 'John Doe', 'Male', 25, '123-456-7890')
(3, 'Bob Johnson', 'Male', 22, '555-123-4567')


In [91]:

cur.execute("""
INSERT INTO "user" (name, sex, age, phone) VALUES
    ('ymy', 'Male', 21, '123-456-7890'),
    ('gxh', 'Male', 26, '234-567-8901'),
    ('czt', 'Male', 18, '345-678-9012');
""")
print("个人信息和虚拟信息已添加。")

个人信息和虚拟信息已添加。


In [92]:

cur.execute("""
SELECT * FROM "user"
WHERE age BETWEEN 20 AND 30 AND name LIKE 'y%'
ORDER BY age DESC;
""")
users_with_surname = cur.fetchall()
print("年龄在 20-30 范围内，名字包含“y”的用户:")
for user in users_with_surname:
    print(user)

年龄在 20-30 范围内，名字包含“y”的用户:
(4, 'ymy', 'Male', 21, '123-456-7890')


In [93]:

cur.execute("""
SELECT AVG(age) AS average_age FROM "user";
""")
average_age = cur.fetchone()[0]
print(f"所有用户的平均年龄: {average_age}")

所有用户的平均年龄: 23.8333333333333333


In [94]:

cur.execute("""
CREATE TABLE IF NOT EXISTS team (
    id SERIAL PRIMARY KEY,
    teamName VARCHAR(100) UNIQUE
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS score (
    id SERIAL PRIMARY KEY,
    teamid VARCHAR(100) REFERENCES team(teamName),
    userid VARCHAR(100) REFERENCES "user"(name),
    score INT
);
""")
print("team 和 score 表已创建。")

team 和 score 表已创建。


In [95]:

cur.execute("""
INSERT INTO team (teamName) VALUES
    ('ECNU'),
    ('Other Team')
ON CONFLICT (teamName) DO NOTHING;  
""")

cur.execute("""
INSERT INTO score (teamid, userid, score) VALUES
    ('ECNU', 'ymy', 85),  
    ('ECNU', 'gxh', 80), 
    ('ECNU', 'czt', 90)   
""")
print("记录已插入到 team 和 score 表中。")

记录已插入到 team 和 score 表中。


In [97]:

cur.execute("""
SELECT u.*
FROM "user" u
JOIN score s ON u.name = s.userid
JOIN team t ON s.teamid = t.teamName
WHERE t.teamName = 'ECNU' AND u.age < 20;
""")
young_users = cur.fetchall()
print("年龄小于 20 的用户:")
for user in young_users:
    print(user)


年龄小于 20 的用户:
(6, 'czt', 'Male', 18, '345-678-9012')
(6, 'czt', 'Male', 18, '345-678-9012')


In [98]:

cur.execute("""
SELECT COALESCE(SUM(s.score), 0) AS total_score
FROM score s
JOIN team t ON s.teamid = t.teamName
WHERE t.teamName = 'ECNU';
""")
total_score = cur.fetchone()[0]
print(f"团队 'ECNU' 的总分: {total_score}")

团队 'ECNU' 的总分: 510


In [99]:

cur.execute("""
DROP TABLE IF EXISTS "user" CASCADE;
""")
print("用户表中的所有记录已删除。")


用户表中的所有记录已删除。
